In [ ]:
# 1. 安裝必要套件
!pip install faiss-cpu sentence-transformers pypdf chardet llama-cpp-python chromadb PyMuPDF --quiet

In [ ]:
!wget -O tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf \
    https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf

In [ ]:
from llama_cpp import Llama

# 載入模型
llm = Llama(model_path="tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf", n_ctx=2048, n_threads=4)
#llm = Llama(model_path="qwen-0.5b-chat.gguf", n_ctx=2048, n_threads=4)
print("模型已載入完成")

In [20]:
from chromadb import Client
import chromadb.utils.embedding_functions as ef

embedding_func = ef.DefaultEmbeddingFunction()
chroma_client = Client()
collection = chroma_client.create_collection(name="docs", embedding_function=embedding_func)
print("已建立 ChromaDB 檢索資料庫")

已建立 ChromaDB 檢索資料庫


In [21]:
# 6. 問答流程 (RAG)
def retrieve(query, top_k=3):
    q_emb = embedding_model.encode([query])
    D, I = index.search(np.array(q_emb).astype('float32'), top_k)
    return [all_chunks[idx] for idx in I[0]]

def rag_qa(query):
    results = collection.query(query_texts=[query], n_results=3)
    context = " ".join(results["documents"][0])
    prompt = f"""使用以下資料回答問題：
{context}

問題：{query}
回答：
"""
    output = llm(prompt, max_tokens=256, stop=["</s>"])
    return output["choices"][0]["text"]


In [22]:
print(rag_qa("簡述國科會經費規定"))

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 41.7MiB/s]
llama_perf_context_print:        load time =    2648.76 ms
llama_perf_context_print: prompt eval time =    2648.30 ms /    55 tokens (   48.15 ms per token,    20.77 tokens per second)
llama_perf_context_print:        eval time =   28756.84 ms /   255 runs   (  112.77 ms per token,     8.87 tokens per second)
llama_perf_context_print:       total time =   31623.18 ms /   310 tokens


國科會經費規定包括兩種方式：

1. 優先契約：國科會會員為各級普通會議委員、總統會委員和國科會推薦委員與國科會會長之間的契約，其中，會長投票決減至最少一個會長。

2. 優先會議：國科會會議會由國科會會長，會臺縣大學大學生會臺縣大學大學生會臺縣大學大學生會臺縣大學大學生會臺縣大學大學生會臺縣大學大學生會臺縣大


In [32]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
import glob
import pypdf

# 建立 embedding 模型
embedder = SentenceTransformer("all-MiniLM-L6-v2")


In [36]:
from google.colab import files
import fitz  # PyMuPDF

uploaded = files.upload()
for filename in uploaded.keys():
    if filename.lower().endswith(".pdf"):
        doc = fitz.open(filename)
        text = ""
        for page in doc:
            text += page.get_text()
    elif filename.lower().endswith(".txt"):
        with open(filename, "r", encoding="utf-8") as f:
            text = f.read()
    else:
        print(f"不支援的檔案格式: {filename}")
        continue

    # 分段加入向量資料庫
    chunks = [text[i:i+500] for i in range(0, len(text), 500)]
    ids = [f"{filename}_{i}" for i in range(len(chunks))]
    collection.add(documents=chunks, ids=ids)
    print(f"已處理並加入知識庫: {filename}")

print("文件已全部加入知識庫")

Saving 國家科學及技術委員會補助國外學者專家來臺從事科技合作研究活動支付費用最高標準表1110830 (3).pdf to 國家科學及技術委員會補助國外學者專家來臺從事科技合作研究活動支付費用最高標準表1110830 (3).pdf
已處理並加入知識庫: 國家科學及技術委員會補助國外學者專家來臺從事科技合作研究活動支付費用最高標準表1110830 (3).pdf
文件已全部加入知識庫


In [38]:
print(rag_qa("簡述國科會經費規定"))

Llama.generate: 22 prefix-match hit, remaining 1467 prompt tokens to eval
llama_perf_context_print:        load time =    2648.76 ms
llama_perf_context_print: prompt eval time =   91963.10 ms /  1467 tokens (   62.69 ms per token,    15.95 tokens per second)
llama_perf_context_print:        eval time =   36228.81 ms /   255 runs   (  142.07 ms per token,     7.04 tokens per second)
llama_perf_context_print:       total time =  128411.77 ms /  1722 tokens


本表為國家科學及技術委員會（以下簡稱本會）補助國外學者專家來臺
與我國學者專家進行合作研究活動。規定，每個月以上至一年者，為衛生福利部
中央健康保險署之「全民健康保險」。本表則依規定做補助，其不足一個月日數
之日支報酬，以月支報酬標準計算，其不足一個月日數之日支報酬，以月支報酬標準
計算，其不足一個月日數之日支報酬，以月支報酬標準計算，其不足
